In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.optimization.QAOA.simulation.QAOARunnerExpValues import QAOARunnerExpValues
from quapopt.optimization.parameter_setting import (ParametersBoundType as PBT)
from quapopt.optimization.parameter_setting.non_adaptive_optimization.SimpleGridOptimizer import SimpleGridOptimizer
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner

### Generate a random Hamiltonian instance

* Like in previous notebooks, we generate a random Hamiltonian instance.
* This time, way larger system sizes.


In [ ]:
coefficients_type = CoefficientsType.DISCRETE
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
localities = (2,)

generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)





## Large-scale p=1 QAOA simulation
* Here we will test simulation of p=1 2-local expected values on large number of qubits.
* NOTE: this runs well on GPUs, so if you don't have CUDA, it might be slow for more than 100 qubits
* To make it more interesting, we will use analytical expressions for optimal betas for each gamma, so we only need to optimize over single parameter. (note that this works only in simulation)

In [ ]:
number_of_qubits = 100
seed_cost_hamiltonian = 0

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True,

                                                                )

if cost_hamiltonian.lowest_energy is None:
    cost_hamiltonian.solve_hamiltonian()

print("Ground state energy:", cost_hamiltonian.lowest_energy)

In [ ]:
number_of_function_calls = 100

#limited range for SK model (could be even smaller in practice)
single_bound = (0, np.pi)

#whether to optimize over betas or use analytical betas;
analytical_betas = True

#let's use grid search optimizer
#just single parameter because we analytically find optimal betas for each gamma
classical_optimizer = SimpleGridOptimizer(parameter_bounds=[(PBT.RANGE, single_bound)] * (1 + (not analytical_betas)),
                                          max_trials=number_of_function_calls)

#uncomment to use COBYQA optimizer instead
# classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[single_bound],
#                                             argument_names = ['Angles-0'],
# optimizer_name='COBYQA',
# optimizer_kwargs=None,
# basinhopping=True,
# basinhopping_kwargs={'niter': 3},
# starting_point=[0.05]
# )

In [ ]:
#if set to None, the simulator will be chosen automatically
simulator = None
precision = np.float32

qaoa_runner_analytical = QAOARunnerExpValues(hamiltonian_representations_cost=[cost_hamiltonian],
                                             store_full_information_in_history=True,
                                             simulator_name=simulator,
                                             precision_float=precision)
qaoa_optimizer_analytical = QAOAOptimizationRunner(qaoa_runner_analytical)

#whether to store all 2-local expected values in memory; This is useful for applying post-processing techniques, such as Quantum Relax & Round
store_correlators = True

best_result_analytical, optimization_res_analytical = qaoa_optimizer_analytical.run_optimization(qaoa_depth=1,
                                                                                                 number_of_function_calls=number_of_function_calls,
                                                                                                 classical_optimizer=classical_optimizer,
                                                                                                 measurement_noise=None,
                                                                                                 show_progress_bar=True,
                                                                                                 verbosity=0,
                                                                                                 analytical_betas=analytical_betas,
                                                                                                 store_correlators=store_correlators)
print(best_result_analytical)


In [ ]:
print(best_result_analytical[0])
#accessing correlators
#Note: THOSE ARE ACTUALLY WEIGHTED CORRELATORS (LOCAL EXPECTED VALUES), i.e., J_{ij} * Z_iZ_j;
#To get ZiZj, please divide by Hamiltonian coefficients
best_correlators = best_result_analytical[0][1][1].correlators
best_energy = best_result_analytical[0][0]

print('best energy:', best_energy)
print(np.round(best_result_analytical[0][1][1].correlators, 3))

### Quantum Relax and Round (QRR) algorithm

We can also apply QRR algorithm to obtain candidate solutions from exp-values of the cost Hamiltonian. The QRR algorithm is based on a matrix of 2-local correlations. The algorithm is described in Ref [1].

[1] Dupont, Maxime, and Bhuvanesh Sundar. "Extending relax-and-round combinatorial optimization solvers with quantum correlations." Physical Review A 109, no. 1 (2024): 012429.

In [ ]:
if store_correlators:
    #This is possible only if we stored correlators
    best_result_qrr, (
        opt_res_qrr, best_bitstrings_qrr, _) = qaoa_optimizer_analytical.apply_QRR_to_optimization_results(
        return_full_history=False,
        show_progress_bar=True)


### Visualization

* Let's take a look at a simple visualization of the NDAR optimization.


In [ ]:
import plotly
import plotly.graph_objects as go

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
df_res_analytical = optimization_res_analytical.trials_dataframe
x_name = f"Angles-0"
if x_name not in df_res_analytical.columns:
    df_res_analytical[x_name] = df_res_analytical['Arguments'].apply(lambda x:x[0])


xs = df_res_analytical[x_name].values
ys = df_res_analytical['FunctionValue']

fig = go.Figure()
fig.add_trace(go.Scatter(x=xs, y=ys, mode='markers', name='QAOA p=1'))
if store_correlators:
    df_res_analytical_qrr = opt_res_qrr.trials_dataframe
    if x_name not in df_res_analytical_qrr.columns:
        df_res_analytical_qrr[x_name] = df_res_analytical_qrr['Angles'].apply(lambda x:x[0])

    xs_qrr = df_res_analytical_qrr[x_name].values
    ys_qrr = opt_res_qrr.trials_dataframe['Energy']
    fig.add_trace(go.Scatter(x=xs_qrr, y=ys_qrr, mode='markers', name='QAOA p=1 + QRR'))

fig.update_layout(
    title=f"QAOA p=1 with analytical betas performance for {number_of_qubits} qubits on {cost_hamiltonian.hamiltonian_class_specifier.get_description_string()}",
    xaxis_title="gamma",
    yaxis_title="Energy",
    template="plotly",
    font=dict(size=14),
    hovermode="x unified")

plotly.offline.plot(fig,
                    filename=f'../temp/p1_simulator_analytical_betas_n={number_of_qubits}.html')
fig.show()